In [1]:
%%time
# Underlying details extracted from nse
# Program takes 5 mins

# STATUS: Completed

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import json
import datetime as datetime
import sys

#*****            Initializations           ****
#_______________________________________________

load_from_disk = True   # For underlying symbol list

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'


#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

#******               Symbols list          ****
#________________________________________________

if load_from_disk:
    try:
        symbols = list(pd.read_csv(datapath+'nse_eq_symbols.csv'))
    except FileNotFoundError as e:
        print('csv file not found')
        sys.exit(0)

#******     Underlying data extraction     *****
#_______________________________________________

def get_underlying(symbol):
    
    '''Gets the underlying data
    Arg: (symbol as string)
    Returns: df as dataframe'''

    # URLs
    url_base1 = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
    url = url_base1 + "get_quote/GetQuote.jsp?symbol="+symbol

    page = requests.get(url).text

    # soup out the json dict
    bs_nse = bs4(page, 'html.parser')
    json_nse = json.loads(bs_nse.find(id='responseDiv').text.strip())

    df = pd.DataFrame.from_dict(json_nse['data'][0], orient='index').T
    
    return df

nse_underlyings = [catch(lambda: get_underlying(symbol)) for symbol in symbols]

# indexed by symbols
df_nse_und = pd.concat(nse_underlyings).set_index('symbol')

# clean up the commas, dashes and empty strs
df_nse_und1 = df_nse_und.replace(',|-' , '', regex=True)
df_nse_und1 = df_nse_und1.replace('', np.nan)

# convert date columns to datetime.date format
filtin = ['Date', 'dt']
filtout = ['isExDateFlag']
date_columns_mask = df_nse_und1.columns.str.contains('|'.join(filtin)) | (df_nse_und1.columns.str.contains('|'.join(filtout)))
dt_col_list = df_nse_und1.loc[:, date_columns_mask].columns.tolist()
dt_col_list = [item for item in dt_col_list if item not in filtout]  #filterout

df_nse_und1.loc[:, dt_col_list] = df_nse_und1.loc [:, dt_col_list].apply( \
                                  pd.to_datetime, errors= 'coerce').applymap(pd.Timestamp.date)

# store data for next program
df_nse_und1.to_pickle(datapath+r'df_underlying.pkl')

Wall time: 5min


In [2]:
df_nse_und1.head()

,pricebandupper,applicableMargin,bcEndDate,totalSellQuantity,adhocMargin,companyName,marketType,exDate,bcStartDate,css_status_desc,...,buyQuantity3,buyQuantity2,buyQuantity1,series,faceValue,buyQuantity5,closePrice,open,isinCode,lastPrice
symbol,,,,,,,,,,,,,,,,,,,,,
BPCL,354.00,19.93,2018-09-11,8945,NaN,Bharat Petroleum Corporation Limited,N,2018-08-31,2018-09-04,Listed,...,NaN,NaN,NaN,EQ,10.00,NaN,322.05,319.60,INE029A01011,319.50
EICHERMOT,26339.85,15.80,2018-08-10,NaN,NaN,Eicher Motors Limited,N,2018-08-02,2018-08-04,Listed,...,NaN,NaN,222,EQ,10.00,NaN,24735.00,24120.00,INE066A01013,24796.10
OFSS,3829.55,12.50,2018-08-14,NaN,NaN,Oracle Financial Services Software Limited,N,2018-08-06,2018-08-08,Listed,...,NaN,NaN,156,EQ,5.00,NaN,3573.45,3494.90,INE881D01027,3580.00
CESC,751.85,13.68,0001-01-01,347,NaN,CESC Limited,N,2018-10-30,0001-01-01,Listed,...,NaN,NaN,NaN,EQ,10.00,NaN,687.05,683.90,INE486A01013,688.00
HCLTECH,1087.60,13.80,0001-01-01,17,NaN,HCL Technologies Limited,N,2018-10-30,0001-01-01,Listed,...,NaN,NaN,NaN,EQ,2.00,NaN,1022.20,997.00,INE860A01027,1022.00
